
Es necesario ajustar nuestras ofertas al perfil del viajero y a la demanda de 
pernoctaciones en las ciudades donde estamos presentes, considerando las cifras oficiales sobre 
procedencia, meses de visita y media de pernoctaciones por ciudad autónoma?

## ETL + EDA Pernoctaciones

Importar librerias

In [1]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import pandas as pd
import locale

Viajeros y pernoctaciones por comunidades autónomas y provincias ENE 1999 ABR 2025

In [2]:

# Leer el archivo CSV
df = pd.read_csv("Viajeros y pernoctaciones por comunidades autónomas y provincias.csv", sep=';', encoding='utf-8-sig')


df_com = df
df_com


,Totales Territoriales,Comunidades y Ciudades Autónomas,Provincias,Viajeros y pernoctaciones,Residencia: Nivel 1,Residencia: Nivel 2,Periodo,Total
0,Total Nacional,NaN,NaN,Viajero,Total,NaN,2025M04,10.018.537
1,Total Nacional,NaN,NaN,Viajero,Total,NaN,2025M03,7.680.722
2,Total Nacional,NaN,NaN,Viajero,Total,NaN,2025M02,6.539.960
3,Total Nacional,NaN,NaN,Viajero,Total,NaN,2025M01,5.776.075
4,Total Nacional,NaN,NaN,Viajero,Total,NaN,2024M12,6.553.489
...,...,...,...,...,...,...,...,...
132715,Total Nacional,19 Melilla,NaN,Pernoctaciones,Total,Residentes en el Extranjero,1999M05,1.305
132716,Total Nacional,19 Melilla,NaN,Pernoctaciones,Total,Residentes en el Extranjero,1999M04,1.668
132717,Total Nacional,19 Melilla,NaN,Pernoctaciones,Total,Residentes en el Extranjero,1999M03,1.673
132718,Total Nacional,19 Melilla,NaN,Pernoctaciones,Total,Residentes en el Extranjero,1999M02,1.487


In [3]:
df_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132720 entries, 0 to 132719
Data columns (total 8 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Totales Territoriales             132720 non-null  object
 1   Comunidades y Ciudades Autónomas  130824 non-null  object
 2   Provincias                        94800 non-null   object
 3   Viajeros y pernoctaciones         132720 non-null  object
 4   Residencia: Nivel 1               132720 non-null  object
 5   Residencia: Nivel 2               88480 non-null   object
 6   Periodo                           132720 non-null  object
 7   Total                             132710 non-null  object
dtypes: object(8)
memory usage: 8.1+ MB


In [4]:

# Convertimos Total a Numerica
# Paso 1: quitar puntos como separadores de miles
df_com["Total"] = df_com["Total"].str.replace(".", "", regex=False)

# Paso 2: reemplazar "0" y "." por NaN
df_com["Total"] = df_com["Total"].replace({"0": np.nan, ".": np.nan})

# Paso 3: convertir a entero que acepte NaN ( Int64) acepta naan
df_com["Total"] = pd.to_numeric(df_com["Total"]).astype("Int64")


# Reemplazar los valores en la columna 'Comunidad_autonoma'
df_com['Comunidades y Ciudades Autónomas'] = df_com['Comunidades y Ciudades Autónomas'].replace({
    '01 AndalucÃ\xada': 'Andalucía',
    '02 AragÃ³n': 'Aragón',
    '03 Asturias, Principado de': 'Asturias',
    '04 Balears, Illes': 'Islas Baleares',
    '05 Canarias': 'Canarias',
    '06 Cantabria': 'Cantabria',
    '07 Castilla y LeÃ³n': 'Castilla y León',
    '08 Castilla - La Mancha': 'Castilla-La Mancha',
    '09 CataluÃ±a': 'Cataluña',
    '10 Comunitat Valenciana': 'Comunidad Valenciana',
    '11 Extremadura': 'Extremadura',
    '12 Galicia': 'Galicia',
    '13 Madrid, Comunidad de': 'Comunidad de Madrid',
    '14 Murcia, RegiÃ³n de': 'Región de Murcia',
    '15 Navarra, Comunidad Foral de': 'Navarra',
    '16 PaÃ\xads Vasco': 'País Vasco',
    '17 Rioja, La': 'La Rioja',
    '18 Ceuta': 'Ceuta',
    '19 Melilla': 'Melilla'
})



# Establecer la configuración regional a español (España)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Extraer el año y el mes con split 'M'
df_com["Año"] = df_com["Periodo"].str.split('M').str[0]
df_com["mes"] = df_com["Periodo"].str.split('M').str[1]

# Convertir a números 
df_com["mes"] = pd.to_numeric(df_com["mes"], errors='coerce')

# Extraer el nombre del mes en español
df_com["Nom_mes"] = pd.to_datetime(df_com["mes"], format='%m').dt.month_name(locale='es_ES.UTF-8')

# Convertimos 'Año' a entero
df_com["Año"] = df_com["Año"].astype(int)


# Borrar columnan innecesarias
df_com.drop(columns=["Periodo"], inplace=True)
df_com.drop(columns=["Residencia: Nivel 1"], inplace=True)

# Reemplazamos valores provincias 

def limpiar_nombre_provincia(valor):
    if isinstance(valor, str):
        # Paso 1: eliminar el código inicial si existe
        if valor[:2].isdigit() and valor[2] == ' ':
            valor = valor[3:]
        # Paso 2: reordenar si contiene coma (ej. 'Rioja, La' → 'La Rioja')
        if ',' in valor:
            partes = valor.split(', ')
            if len(partes) == 2:
                valor = f"{partes[1]} {partes[0]}"
    return valor

# Aplicar la función a la columna correspondiente (ej. 'Provincias')
df_com["Provincias"] = df_com["Provincias"].apply(limpiar_nombre_provincia)




In [5]:
df_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132720 entries, 0 to 132719
Data columns (total 9 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Totales Territoriales             132720 non-null  object
 1   Comunidades y Ciudades Autónomas  130824 non-null  object
 2   Provincias                        94800 non-null   object
 3   Viajeros y pernoctaciones         132720 non-null  object
 4   Residencia: Nivel 2               88480 non-null   object
 5   Total                             132215 non-null  Int64 
 6   Año                               132720 non-null  int64 
 7   mes                               132720 non-null  int64 
 8   Nom_mes                           132720 non-null  object
dtypes: Int64(1), int64(2), object(6)
memory usage: 9.2+ MB


In [6]:
df_com

,Totales Territoriales,Comunidades y Ciudades Autónomas,Provincias,Viajeros y pernoctaciones,Residencia: Nivel 2,Total,Año,mes,Nom_mes
0,Total Nacional,NaN,NaN,Viajero,NaN,10018537,2025,4,Abril
1,Total Nacional,NaN,NaN,Viajero,NaN,7680722,2025,3,Marzo
2,Total Nacional,NaN,NaN,Viajero,NaN,6539960,2025,2,Febrero
3,Total Nacional,NaN,NaN,Viajero,NaN,5776075,2025,1,Enero
4,Total Nacional,NaN,NaN,Viajero,NaN,6553489,2024,12,Diciembre
...,...,...,...,...,...,...,...,...,...
132715,Total Nacional,Melilla,NaN,Pernoctaciones,Residentes en el Extranjero,1305,1999,5,Mayo
132716,Total Nacional,Melilla,NaN,Pernoctaciones,Residentes en el Extranjero,1668,1999,4,Abril
132717,Total Nacional,Melilla,NaN,Pernoctaciones,Residentes en el Extranjero,1673,1999,3,Marzo
132718,Total Nacional,Melilla,NaN,Pernoctaciones,Residentes en el Extranjero,1487,1999,2,Febrero


In [7]:
df_com["Viajeros y pernoctaciones"].unique()

array(['Viajero', 'Pernoctaciones'], dtype=object)

Viajeros y pernoctaciones por comunidades autónomas y provincias ENE 1999 ABR 2025

In [8]:
# Leer el archivo CSV
df = pd.read_csv("Viajeros y pernoctaciones según país de residencia del viajero.csv", sep=';', encoding='utf-8-sig')

df_resid = df


df_resid

,RESIDENCIA/ORIGEN,Países,Viajeros y pernoctaciones,Periodo,Total
0,Total,NaN,Viajero,2025M04,10.018.537
1,Total,NaN,Viajero,2025M03,7.680.722
2,Total,NaN,Viajero,2025M02,6.539.960
3,Total,NaN,Viajero,2025M01,5.776.075
4,Total,NaN,Viajero,2024M12,6.553.489
...,...,...,...,...,...
20219,Asia (sin Japón),NaN,Pernoctaciones,1999M05,25.427
20220,Asia (sin Japón),NaN,Pernoctaciones,1999M04,26.878
20221,Asia (sin Japón),NaN,Pernoctaciones,1999M03,24.271
20222,Asia (sin Japón),NaN,Pernoctaciones,1999M02,18.530


In [9]:
df_resid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20224 entries, 0 to 20223
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   RESIDENCIA/ORIGEN          20224 non-null  object
 1   Países                     17064 non-null  object
 2   Viajeros y pernoctaciones  20224 non-null  object
 3   Periodo                    20224 non-null  object
 4   Total                      19082 non-null  object
dtypes: object(5)
memory usage: 790.1+ KB


In [10]:
# Paso 1: quitar puntos como separadores de miles
df_resid["Total"] = df_resid["Total"].str.replace(".", "", regex=False)

# Paso 2: reemplazar "0" y "." por NaN
df_resid["Total"] = df_resid["Total"].replace({"0": np.nan, ".": np.nan})

# Paso 3: convertir a entero que acepte NaN
df_resid["Total"] = pd.to_numeric(df_resid["Total"]).astype("Int64")

# Establecer la configuración regional a español (España)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Extraer el año y el mes con split 'M'
df_resid["Año"] = df_resid["Periodo"].str.split('M').str[0]
df_resid["mes"] = df_resid["Periodo"].str.split('M').str[1]

# Convertir a números
df_resid["mes"] = pd.to_numeric(df_resid["mes"], errors='coerce')

# Extraer el nombre del mes en español
df_resid["Nom_mes"] = pd.to_datetime(df_resid["mes"], format='%m').dt.month_name(locale='es_ES.UTF-8')

# Convertimos 'Año' a entero
df_resid["Año"] = df_resid["Año"].astype(int)

# Borrar columnan innecesarias
df_resid.drop(columns=["Periodo"], inplace=True)



In [11]:
df_resid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20224 entries, 0 to 20223
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   RESIDENCIA/ORIGEN          20224 non-null  object
 1   Países                     17064 non-null  object
 2   Viajeros y pernoctaciones  20224 non-null  object
 3   Total                      18860 non-null  Int64 
 4   Año                        20224 non-null  int64 
 5   mes                        20224 non-null  int64 
 6   Nom_mes                    20224 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [12]:
df_resid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20224 entries, 0 to 20223
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   RESIDENCIA/ORIGEN          20224 non-null  object
 1   Países                     17064 non-null  object
 2   Viajeros y pernoctaciones  20224 non-null  object
 3   Total                      18860 non-null  Int64 
 4   Año                        20224 non-null  int64 
 5   mes                        20224 non-null  int64 
 6   Nom_mes                    20224 non-null  object
dtypes: Int64(1), int64(2), object(4)
memory usage: 1.1+ MB


Viajeros y pernoctaciones por puntos turísticos ENE 2005 Abr 2025

In [13]:
# Leer el archivo CSV
df = pd.read_csv("Viajeros y pernoctaciones por puntos turísticos.csv", sep=';', encoding='utf-8-sig')

df_ciud = df


df_ciud

,Puntos turísticos,Viajeros y pernoctaciones,Residencia,Periodo,Total
0,01059 Vitoria-Gasteiz,Viajero,Residentes en España,2025M04,24.457
1,01059 Vitoria-Gasteiz,Viajero,Residentes en España,2025M03,23.347
2,01059 Vitoria-Gasteiz,Viajero,Residentes en España,2025M02,18.341
3,01059 Vitoria-Gasteiz,Viajero,Residentes en España,2025M01,19.024
4,01059 Vitoria-Gasteiz,Viajero,Residentes en España,2024M12,23.125
...,...,...,...,...,...
134683,50297 Zaragoza,Pernoctaciones,Residentes en el Extranjero,2005M05,20.698
134684,50297 Zaragoza,Pernoctaciones,Residentes en el Extranjero,2005M04,20.003
134685,50297 Zaragoza,Pernoctaciones,Residentes en el Extranjero,2005M03,21.156
134686,50297 Zaragoza,Pernoctaciones,Residentes en el Extranjero,2005M02,13.372


In [14]:
df_ciud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134688 entries, 0 to 134687
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Puntos turísticos          134688 non-null  object
 1   Viajeros y pernoctaciones  134688 non-null  object
 2   Residencia                 134688 non-null  object
 3   Periodo                    134688 non-null  object
 4   Total                      99134 non-null   object
dtypes: object(5)
memory usage: 5.1+ MB


In [15]:




# Establecer la configuración regional a español (España)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Extraer el año y el mes con split 'M'
df_ciud["Año"] = df_ciud["Periodo"].str.split('M').str[0]
df_ciud["mes"] = df_ciud["Periodo"].str.split('M').str[1]

# Convertir a números
df_ciud["mes"] = pd.to_numeric(df_ciud["mes"], errors='coerce')

# Extraer el nombre del mes en español
df_ciud["Nom_mes"] = pd.to_datetime(df_ciud["mes"], format='%m').dt.month_name(locale='es_ES.UTF-8')

# Convertimos 'Año' a entero
df_ciud["Año"] = df_ciud["Año"].astype(int)

# Borrar columna innecesaria
df_ciud.drop(columns=["Periodo"], inplace=True)

# Renombrar columna
df_ciud.rename(columns={"Puntos turísticos": "Ciudad"}, inplace=True)

# Quitar el código inicial (5 números + 1 espacio)
df_ciud["Ciudad"] = df_ciud["Ciudad"].str[6:]

# Reemplazos manuales para nombres especiales
reemplazos_ciudades = {
    "Coruña, A": "A Coruña",
    "Hospitalet de Llobregat, L'": "L'Hospitalet de Llobregat"
    
}

df_ciud["Ciudad"] = df_ciud["Ciudad"].replace(reemplazos_ciudades)

# Paso 1: quitar puntos como separadores de miles
df_ciud["Total"] = df_ciud["Total"].str.replace(".", "", regex=False)


# Paso 2: reemplazar "0" y "." por NaN
df_ciud["Total"] = df_ciud["Total"].replace({"0": np.nan, ".": np.nan})

# Paso 3: convertir a entero que acepte NaN
df_ciud["Total"] = pd.to_numeric(df_ciud["Total"]).astype("Int64")



Exportar df como CSV

In [ ]:
# Exportar df_com limpio
#df_com.to_csv("df_com_limpio.csv", index=False, encoding="utf-8-sig")

# Exportar df_resid limpio
#df_resid.to_csv("df_resid_limpio.csv", index=False, encoding="utf-8-sig")

# Exportar df_ciud limpio
#df_ciud.to_csv("df_ciud_limpio.csv", index=False, encoding="utf-8-sig")
